## 简单操作
安装jupyter notebook相关插件
pip install jupyter_nbextensions_configurator jupyter_contrib_nbextensions -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [38]:
import pandas as pd
df = pd.DataFrame({'A1': [1, 2, 2], 'A2': [4, 3, 6],
                   'A3': [7, 2, 5]}, index=['一', '二', '三'])
df

,A1,A2,A3
一,1,4,7
二,2,3,2
三,2,6,5


### stack和unstack的使用--常用的股票中

In [5]:
df1=df.stack()
df1

一  A1    1
   A2    4
   A3    7
二  A1    2
   A2    3
   A3    2
三  A1    2
   A2    6
   A3    5
dtype: int64

In [6]:
df1.unstack()

,A1,A2,A3
一,1,4,7
二,2,3,2
三,2,6,5


### stack和set_index结合使用

In [1]:
import tushare as ts
pro=ts.pro_api('5f2dc9d17ebed3601a4fbea74148a25067d067138bf88515693e9b7d')
df=pro.stock_basic(exchange='',list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
df.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,国农科技,深圳,生物制药,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427


In [17]:
df2=df.set_index(['ts_code','name'])
df2.head()

,,symbol,area,industry,list_date
ts_code,name,,,,
000001.SZ,平安银行,000001,深圳,银行,19910403
000002.SZ,万科A,000002,深圳,全国地产,19910129
000004.SZ,国农科技,000004,深圳,生物制药,19910114
000005.SZ,世纪星源,000005,深圳,环境保护,19901210
000006.SZ,深振业A,000006,深圳,区域地产,19920427


In [20]:
df3=df2.stack()
df3.head(10)

ts_code    name           
000001.SZ  平安银行  symbol         000001
                 area               深圳
                 industry           银行
                 list_date    19910403
000002.SZ  万科A   symbol         000002
                 area               深圳
                 industry         全国地产
                 list_date    19910129
000004.SZ  国农科技  symbol         000004
                 area               深圳
dtype: object

In [22]:
df4=df3.reset_index()
df4.head()

,ts_code,name,level_2,0
0,000001.SZ,平安银行,symbol,000001
1,000001.SZ,平安银行,area,深圳
2,000001.SZ,平安银行,industry,银行
3,000001.SZ,平安银行,list_date,19910403
4,000002.SZ,万科A,symbol,000002


### 长宽表转换

In [23]:
df.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,国农科技,深圳,生物制药,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427


## 数据分组和透视

### 数据分组--groupby

In [4]:
pro=ts.pro_api('5f2dc9d17ebed3601a4fbea74148a25067d067138bf88515693e9b7d')
df=pro.stock_basic(exchange='',list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')
df['list_date']=df['list_date'].astype('datetime64')
df.head(10)

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,1991-04-03
1,000002.SZ,000002,万科A,深圳,全国地产,主板,1991-01-29
2,000004.SZ,000004,国农科技,深圳,生物制药,主板,1991-01-14
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,1990-12-10
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,1992-04-27
5,000007.SZ,000007,全新好,深圳,酒店餐饮,主板,1992-04-13
6,000008.SZ,000008,神州高铁,北京,运输设备,主板,1992-05-07
7,000009.SZ,000009,中国宝安,深圳,综合类,主板,1991-06-25
8,000010.SZ,000010,*ST美丽,深圳,建筑工程,主板,1995-10-27
9,000011.SZ,000011,深物业A,深圳,区域地产,主板,1992-03-30


In [20]:
df.info()   #有部分缺失值，但是不影响，因为groupby会自动忽略缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Data columns (total 7 columns):
ts_code      3750 non-null object
symbol       3750 non-null object
name         3750 non-null object
area         3748 non-null object
industry     3748 non-null object
market       3750 non-null object
list_date    3750 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 205.2+ KB


#### 生成一个groupby对象

In [5]:
df.head()

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,1991-04-03
1,000002.SZ,000002,万科A,深圳,全国地产,主板,1991-01-29
2,000004.SZ,000004,国农科技,深圳,生物制药,主板,1991-01-14
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,1990-12-10
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,1992-04-27


In [9]:
#按market板块进行分组,,生成一个groupby对象df1，，对其可进行相关描述
df1=df.groupby('market')
df1

In [29]:
#通过定义函数的方式，进行排序

def func(x):  
    return x.sort_values(by='list_date',ascending=False)


df1.apply(func)

ts_code  symbol  name area industry market  list_date
market                                                              
中小板    1398  002972.SZ  002972  N科安达   深圳     运输设备    中小板 2019-12-27
       1397  002970.SZ  002970  锐明技术   深圳     通信设备    中小板 2019-12-17
       1395  002968.SZ  002968   新大正   重庆     房产服务    中小板 2019-12-03
       1396  002969.SZ  002969  嘉美包装   安徽     广告包装    中小板 2019-12-02
       1394  002967.SZ  002967  广电计量   广东      综合类    中小板 2019-11-08
       1391  002963.SZ  002963   豪尔赛   北京     装修装饰    中小板 2019-10-28
       1392  002965.SZ  002965  祥鑫科技   广东     汽车配件    中小板 2019-10-25
       1390  002962.SZ  002962  五方光电   湖北      元器件    中小板 2019-09-17
       1389  002961.SZ  002961  瑞达期货   福建     多元金融    中小板 2019-09-05
       1399  003816.SZ  003816  中国广核   深圳     新型电力    中小板 2019-08-26
       1387  002959.SZ  002959  小熊电器   广东     家用电器    中小板 2019-08-23
       1388  002960.SZ  002960  青鸟消防   河北     专用机械    中小板 2019-08-09
       1393  002966.SZ  002966  苏州银行   江苏       银行    中小板 2019-08-02
       1385  002957.SZ  002957  科瑞技术   深圳     专用机械    中小板 2019-07-26
       1384  002956.SZ  002956  西麦食品   广西       食品    中小板 2019-06-19
       1383  002955.SZ  002955  鸿合科技   北京     IT设备    中小板 2019-05-23
       1382  002953.SZ  002953  日丰股份   广东     电气设备    中小板 2019-05-09
       1381  002952.SZ  002952  亚世光电   辽宁      元器件    中小板 2019-03-28
       1386  002958.SZ  002958  青农商行   山东       银行    中小板 2019-03-26
       1380  002951.SZ  002951  金时科技   四川     广告包装    中小板 2019-03-15
       1379  002950.SZ  002950  奥美医疗   湖北     医疗保健    中小板 2019-03-11
       1378  002949.SZ  002949  华阳国际   深圳     建筑工程    中小板 2019-02-26
       1376  002947.SZ  002947   恒铭达   江苏      元器件    中小板 2019-02-01
       1375  002946.SZ  002946   新乳业   四川      乳制品    中小板 2019-01-25
       1374  002945.SZ  002945  华林证券   西藏       证券    中小板 2019-01-17
       1377  002948.SZ  002948  青岛银行   山东       银行    中小板 2019-01-16
       1372  002942.SZ  002942  新农股份   浙江     农药化肥    中小板 2018-12-05
       1373  002943.SZ  002943  宇晶股份   湖南     机床制造    中小板 2018-11-29
       1371  002941.SZ  002941  新疆交建   新疆     建筑工程    中小板 2018-11-28
       1369  002939.SZ  002939  长城证券   深圳       证券    中小板 2018-10-26
...                ...     ...   ...  ...      ...    ...        ...
科创板    3718  688116.SH  688116  天奈科技   江苏     化工原料    科创板 2019-09-25
       3726  688168.SH  688168   安博通   北京     软件服务    科创板 2019-09-06
       3739  688321.SH  688321  微芯生物   深圳     化学制药    科创板 2019-08-12
       3714  688099.SH  688099  晶晨股份   上海      半导体    科创板 2019-08-08
       3727  688188.SH  688188  柏楚电子   上海     软件服务    科创板 2019-08-08
       3683  688005.SH  688005  容百科技   浙江     电气设备    科创板 2019-07-22
       3684  688006.SH  688006  杭可科技   浙江     电气设备    科创板 2019-07-22
       3740  688333.SH  688333   铂力特   陕西     机械基件    科创板 2019-07-22
       3681  688002.SH  688002  睿创微纳   山东     通信设备    科创板 2019-07-22
       3682  688003.SH  688003  天准科技   江苏     专用机械    科创板 2019-07-22
       3697  688022.SH  688022  瀚川智能   江苏     专用机械    科创板 2019-07-22
       3703  688033.SH  688033  天宜上佳   北京     运输设备    科创板 2019-07-22
       3747  688388.SH  688388  嘉元科技   广东      元器件    科创板 2019-07-22
       3701  688029.SH  688029  南微医学   江苏     医疗保健    科创板 2019-07-22
       3685  688007.SH  688007  光峰科技   深圳      元器件    科创板 2019-07-22
       3687  688009.SH  688009  中国通号   北京     运输设备    科创板 2019-07-22
       3708  688066.SH  688066  航天宏图   北京     软件服务    科创板 2019-07-22
       3686  688008.SH  688008  澜起科技   上海      半导体    科创板 2019-07-22
       3694  688019.SH  688019  安集科技   上海      半导体    科创板 2019-07-22
       3688  688010.SH  688010  福光股份   福建     电器仪表    科创板 2019-07-22
       3711  688088.SH  688088  虹软科技   浙江     软件服务    科创板 2019-07-22
       3689  688011.SH  688011  新光光电  黑龙江      元器件    科创板 2019-07-22
       3690  688012.SH  688012  中微公司   上海     专用机械    科创板 2019-07-22
       3695  688020.SH  688020  方邦股份   广东      元器件    科创板 2019-07-22
       3700  688028.SH  688028   沃尔德   北京     专用机械    科创板 2019-07-22
       3691  

#### 进行数量统计描述或求和
求和等运算类的描述，要求依据对象必须为数值型的
像‘industry’等类型的可进行统计
groupby对象是一个可迭代对象，通过for循环进行迭代

In [ ]:
#进行迭代
for i in df1:
    print(i)

In [20]:
#转换成列表的形式,里面包含4个元祖
df2=list(df1)
df2

[('中小板',         ts_code  symbol   name area industry market  list_date
  459   002001.SZ  002001    新和成   浙江     化学制药    中小板 2004-06-25
  460   002002.SZ  002002   鸿达兴业   江苏     化工原料    中小板 2004-06-25
  461   002003.SZ  002003   伟星股份   浙江       服饰    中小板 2004-06-25
  462   002004.SZ  002004   华邦健康   重庆     化学制药    中小板 2004-06-25
  463   002005.SZ  002005  *ST德豪   安徽     家用电器    中小板 2004-06-25
  464   002006.SZ  002006   精功科技   浙江     专用机械    中小板 2004-06-25
  465   002007.SZ  002007   华兰生物   河南     生物制药    中小板 2004-06-25
  466   002008.SZ  002008   大族激光   深圳     电器仪表    中小板 2004-06-25
  467   002009.SZ  002009   天奇股份   江苏     工程机械    中小板 2004-06-29
  468   002010.SZ  002010   传化智联   浙江     仓储物流    中小板 2004-06-29
  469   002011.SZ  002011   盾安环境   浙江     工程机械    中小板 2004-07-05
  470   002012.SZ  002012   凯恩股份   浙江       造纸    中小板 2004-07-05
  471   002013.SZ  002013   中航机电   湖北       航空    中小板 2004-07-05
  472   002014.SZ  002014   永新股份   安徽       塑料    中小板 2004-07-08
  473   002015.SZ 

In [23]:
len(df2)

4

In [21]:
#转换成字典的形式的形式,需要先转换成列表，再转换成字典
df3=dict(list(df1))
df3


{'中小板':         ts_code  symbol   name area industry market  list_date
 459   002001.SZ  002001    新和成   浙江     化学制药    中小板 2004-06-25
 460   002002.SZ  002002   鸿达兴业   江苏     化工原料    中小板 2004-06-25
 461   002003.SZ  002003   伟星股份   浙江       服饰    中小板 2004-06-25
 462   002004.SZ  002004   华邦健康   重庆     化学制药    中小板 2004-06-25
 463   002005.SZ  002005  *ST德豪   安徽     家用电器    中小板 2004-06-25
 464   002006.SZ  002006   精功科技   浙江     专用机械    中小板 2004-06-25
 465   002007.SZ  002007   华兰生物   河南     生物制药    中小板 2004-06-25
 466   002008.SZ  002008   大族激光   深圳     电器仪表    中小板 2004-06-25
 467   002009.SZ  002009   天奇股份   江苏     工程机械    中小板 2004-06-29
 468   002010.SZ  002010   传化智联   浙江     仓储物流    中小板 2004-06-29
 469   002011.SZ  002011   盾安环境   浙江     工程机械    中小板 2004-07-05
 470   002012.SZ  002012   凯恩股份   浙江       造纸    中小板 2004-07-05
 471   002013.SZ  002013   中航机电   湖北       航空    中小板 2004-07-05
 472   002014.SZ  002014   永新股份   安徽       塑料    中小板 2004-07-08
 473   002015.SZ  002015   协鑫能科  

4

In [22]:
len(df3)
df3.keys()

4

dict_keys(['中小板', '主板', '创业板', '科创板'])

In [28]:
df1.count()['industry']

market
中小板     941
主板     1949
创业板     788
科创板      70
Name: industry, dtype: int64

In [32]:
df1.size()   #后面不可以再跟其他统计依据

market
中小板     941
主板     1951
创业板     788
科创板      70
dtype: int64

#### aggregate函数的使用

In [51]:
#层次描述，先分组，再对分组后的pe_2017进行count，size和sum描述，
#size和count的不同在于，size可以统计NAN，而count不统计
df1=df.groupby(['market','industry'])
df1.aggregate(['count','sum'])['pe_2017'].head()

count  sum
market industry            
中小板    IT设备         13  130
       专用机械         28  280
       中成药          22  220
       乳制品           7   70
       互联网          27  270

In [31]:
#查看所有行业中，不同板块的数量，所有行业中，中小板块的有941个公司
df1.count()['industry'] 

market
中小板     941
主板     1949
创业板     788
科创板      70
Name: industry, dtype: int64

### groupby的其他分组类型

#### 以字典的方式为分组依据--跨列axis=1
以字典里新传入的列索引为分组依据，列索引名相同的为一组，如列索引名为red的为一组

In [37]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2, 2], 'b': [4, 3, 6],
                   'c': [7, 2, 5], 'd': [0, 2, 4]}, index=['一', '二', '三'])
df

,a,b,c,d
一,1,4,7,0
二,2,3,2,2
三,2,6,5,4


In [38]:
dic={'a':'red','b':'blue','c':'blue','d':'red','f':'red'}
df.groupby(dic,axis=1).size()

blue    2
red     2
dtype: int64

#### 数据类型为依据--跨列-axis=1
以每列的数据类型为分类依据，相同类型的为一组，如int类的一组，float为一组，object为一组……

In [40]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'a': ['name', 'age', 2], 'b': [4, 3, 6],
                   'c': [7.1, 2.2, np.nan], 'd': [0, 2, 4], 'e': [4.3, 4.6, 5]}, index=['一', '二', '三'])
df

,a,b,c,d,e
一,name,4,7.1,0,4.3
二,age,3,2.2,2,4.6
三,2,6,NaN,4,5.0


In [41]:
df.dtypes

a     object
b      int64
c    float64
d      int64
e    float64
dtype: object

In [42]:
df.groupby(df.dtypes,axis=1).size()

int64      2
float64    2
object     1
dtype: int64

In [43]:
df.groupby(df.dtypes,axis=1).sum()

,int64,float64,object
一,4,11.4,name
二,5,6.8,age
三,10,5.0,2


####  自定义函数进行分组
自定义的函数有一定要求，即传入的参数必须是--行或者列---索引名
通过设置axis=的值进一步确定传入的参数，axis=0以行索引名为参数，axis=1以列索引名为参数

In [53]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2, 2, 2], 'bb': [4, 3, 6, 1],
                   'cc': [7, 2, 5, 3], 'ddd': [0, 2, 4, 6]}, index=['age', 'ne', 'how', 'hhhh'])
df

,a,bb,cc,ddd
age,1,4,7,0
ne,2,3,2,2
how,2,6,5,4
hhhh,2,1,3,6


In [54]:
def func(x):
    return len(x)

In [55]:
#以行索引名的长度为分类依据，长度相同的为一组
df.groupby(func,axis=0).size()  

2    1
3    2
4    1
dtype: int64

In [ ]:
#以列索引名的长度为分类依据，长度相同的为一组

In [56]:
df.groupby(func,axis=1).size()  

1    1
2    2
3    1
dtype: int64

#### 不同分组依据间的混合使用
字典方式的，函数，数据类型……

In [57]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2, 2, 2], 'bb': [4, 3, 6, 1],
                   'cc': [7, 2, 5, 3], 'ddd': [0, 2, 4, 6]}, index=['age', 'ne', 'how', 'hhhh'])
df

,a,bb,cc,ddd
age,1,4,7,0
ne,2,3,2,2
how,2,6,5,4
hhhh,2,1,3,6


In [69]:
dic={'a':'red','bb':'blue','cc':'red','ddd':'blue','f':'red'}
def func(x):
    return len(x)

In [70]:
df.groupby(func,axis=1).min()


,1,2,3
age,1,4,0
ne,2,2,2
how,2,5,4
hhhh,2,1,6


In [71]:
df.groupby(dic,axis=1).min()

,blue,red
age,0,1
ne,2,2
how,4,2
hhhh,1,2


In [73]:
df.groupby([func,dic],axis=1).min()

1    2        3
     red blue red blue
age    1    4   7    0
ne     2    3   2    2
how    2    6   5    4
hhhh   2    1   3    6

In [74]:
df.groupby([dic,func],axis=1).min()

blue    red   
        2  3   1  2
age     4  0   1  7
ne      3  2   2  2
how     6  4   2  5
hhhh    1  6   2  3

In [72]:
df.groupby([func,dic],axis=1).size()

1  red     1
2  blue    1
   red     1
3  blue    1
dtype: int64

### transform（）
gropuby 结合transform类似于groupby后，进行聚合

In [12]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2, 2, 2], 'bb': [4, 3, 6, 1],
                   'cc': [7, 2, 5, 3], 'ddd': [0, 2, 4, 6]}, index=['age', 'ne', 'how', 'hhhh'])
df

,a,bb,cc,ddd
age,1,4,7,0
ne,2,3,2,2
how,2,6,5,4
hhhh,2,1,3,6


In [13]:
df.groupby('a').sum()

,bb,cc,ddd
a,,,
1,4,7,0
2,10,10,12


In [14]:
df1=df.groupby('a').transform(sum)
df1

,bb,cc,ddd
age,4,7,0
ne,10,10,12
how,10,10,12
hhhh,10,10,12


#### add_prefix('sum_')
将新生成的的数据和原始数据进行区分

In [15]:
df1=df.groupby('a').transform(sum).add_prefix('sum_')
df1

,sum_bb,sum_cc,sum_ddd
age,4,7,0
ne,10,10,12
how,10,10,12
hhhh,10,10,12


####  实现merge合并的功能

In [11]:

df[df1.columns]=df1
df

,a,bb,cc,ddd,sum_bb,sum_cc,sum_ddd
age,1,4,7,0,4,7,0
ne,2,3,2,2,10,10,12
how,2,6,5,4,10,10,12
hhhh,2,1,3,6,10,10,12


### 索引级别分组---MultiIndex
可以通过修改index和columns参数，实现层次化索引的目的
MultiIndex下有很多函数可以使用
from_arrays（）---传入一个array数组  [[],[],[],……]，内部的[]表示一个个分类，通过name参数，对分类进行命名
from_tuples（）----传入一个元祖列表   [（）,（）,（）,……] 


In [75]:
help(pd.MultiIndex)

Help on class MultiIndex in module pandas.core.indexes.multi:

class MultiIndex(pandas.core.indexes.base.Index)
 |  MultiIndex(levels=None, codes=None, sortorder=None, names=None, dtype=None, copy=False, name=None, verify_integrity=True, _set_identity=True)
 |  
 |  A multi-level, or hierarchical, index object for pandas objects.
 |  
 |  Parameters
 |  ----------
 |  levels : sequence of arrays
 |      The unique labels for each level.
 |  codes : sequence of arrays
 |      Integers for each level designating which label at each location.
 |  
 |      .. versionadded:: 0.24.0
 |  labels : sequence of arrays
 |      Integers for each level designating which label at each location.
 |  
 |      .. deprecated:: 0.24.0
 |          Use ``codes`` instead
 |  sortorder : optional int
 |      Level of sortedness (must be lexicographically sorted by that
 |      level).
 |  names : optional sequence of objects
 |      Names for each of the index levels. (name is accepted for compat).
 |  copy 

#### ---from_arrays()

In [126]:
arrays = [['kg', 'kg','pond','pond','mkk'], ['red', 'blue', 'red', 'blue','blue']]
column=pd.MultiIndex.from_arrays(arrays, names=('weight', 'color'))
column

MultiIndex(levels=[['kg', 'mkk', 'pond'], ['blue', 'red']],
           codes=[[0, 0, 2, 2, 1], [1, 0, 1, 0, 0]],
           names=['weight', 'color'])

In [127]:
#要生成四列数据，行数无所谓，与groupby中按两个字典进行分组得到的结果

df=pd.DataFrame(np.random.randint(10,100,15).reshape(3,5),columns=column,index=['一', '二', '三'])
df

weight  kg      pond       mkk
color  red blue  red blue blue
一       41   39   95   74   16
二       73   81   40   61   60
三       36   50   51   83   88

#### 与groupby的对比
虽然格式相似，但是完全不一样：
首先，.MultiIndex显示的是分组后的一个结果，里面的数据均为原数据，而groupby后的结果需要结合统计分析才能以表格的形式展现出来
其次，MultiIndex后得到的表格有名字，而groupby没有

In [100]:
df = pd.DataFrame({'a': [2, 4, 2], 'b': [4, 3, 6],
                   'c': [7.1, 2.2, np.nan], 'd': [0, 2, 4],'e':[2,4,6]}, index=['一', '二', '三'])
df

,a,b,c,d,e
一,2,4,7.1,0,2
二,4,3,2.2,2,4
三,2,6,NaN,4,6


In [101]:
dic1 = {'a': 'red', 'b': 'blue', 'c': 'red', 'd': 'blue','e':'blue'}
dic2 = {'a': 1, 'b': 3, 'c': 2, 'd': 2, 'e': 2}
df.groupby([dic2, dic1], axis=1).min()

1    2         3
   red blue  red blue
一  2.0  0.0  7.1  4.0
二  4.0  2.0  2.2  3.0
三  2.0  4.0  NaN  6.0

#### --from_tuples()

随机生成一个DataFram，生成的列数要和from_tuples中传入的（）数保持一致

In [107]:
a=pd.MultiIndex.from_tuples([('weight','kg'),('weight','pound'),('color','red'),('color','blue')])
a

MultiIndex(levels=[['color', 'weight'], ['blue', 'kg', 'pound', 'red']],
           codes=[[1, 1, 0, 0], [1, 2, 3, 0]])

In [109]:
df=pd.DataFrame(np.random.randint(10,100,12).reshape(3,4),columns=a,index=['一', '二', '三'])
df

weight       color     
      kg pound   red blue
一     12    81    49   84
二     86    85    14   43
三     63    12    16   56

已有df，传入新的columns时，若df有5列，则from_tuples中需要传入5个（）

In [110]:
df = pd.DataFrame({'a': [2, 4, 2], 'b': [4, 3, 6],
                   'c': [7.1, 2.2, np.nan], 'd': [0, 2, 4],'e':[2,4,6]}, index=['一', '二', '三'])
df

,a,b,c,d,e
一,2,4,7.1,0,2
二,4,3,2.2,2,4
三,2,6,NaN,4,6


In [120]:
a = pd.MultiIndex.from_tuples([('weight', 'kg'), ('weight', 'pound'),
                              ('color', 'red'), ('color', 'blue'),('color', 'blue')])
a

MultiIndex(levels=[['color', 'weight'], ['blue', 'kg', 'pound', 'red']],
           codes=[[1, 1, 0, 0, 0], [1, 2, 3, 0, 0]])

In [121]:
df.columns=a
df

weight       color          
      kg pound   red blue blue
一      2     4   7.1    0    2
二      4     3   2.2    2    4
三      2     6   NaN    4    6

#### 与之前的层次化索引对比

In [22]:
import pandas as pd 
import numpy as np 
column=[['weight','weight','weight','color','color'],['kg','pond', 'mkk','red', 'blue']]
df = pd.DataFrame([[2, 4, 2,2,5],[4, 3, 6,77,9],
                   [2, 4, 6,8,9]])
df.columns=column

df

weight          color     
      kg pond mkk   red blue
0      2    4   2     2    5
1      4    3   6    77    9
2      2    4   6     8    9

In [23]:

df.columns.names=['一级','二级']
df

一级 weight          color     
二级     kg pond mkk   red blue
0       2    4   2     2    5
1       4    3   6    77    9
2       2    4   6     8    9

In [21]:
df.columns            

MultiIndex(levels=[['color', 'weight'], ['blue', 'kg', 'mkk', 'pond', 'red']],
           codes=[[1, 1, 1, 0, 0], [1, 3, 2, 4, 0]],
           names=['一级', '二级'])

#### 修改index索引

In [25]:
df = pd.DataFrame([[2, 4, 2,2,5],[4, 3, 6,77,9],
                   [2, 4, 6,8,9],[22, 53, 6,77,9]])
df

,0,1,2,3,4
0,2,4,2,2,5
1,4,3,6,77,9
2,2,4,6,8,9
3,22,53,6,77,9


使用MultiIndex方法

In [31]:
inde=pd.MultiIndex.from_tuples([('上半年','第一季度'),('上半年','第二季度'),('下半年','第三季度'),('下半年','第四季度')],names=['年度','季度'])
df.index=inde

df

0   1  2   3  4
年度  季度                    
上半年 第一季度   2   4  2   2  5
    第二季度   4   3  6  77  9
下半年 第三季度   2   4  6   8  9
    第四季度  22  53  6  77  9

用普通方法

In [ ]:
df = pd.DataFrame([[2, 4, 2,2,5],[4, 3, 6,77,9],
                   [2, 4, 6,8,9],[22, 53, 6,77,9]])
df

In [30]:
inde=[['上半年','上半年','下半年','下半年'],['第一季度','第二季度','第三季度','第四季度']]
df.index=inde
df.index.names=['年度','季度']
df

0   1  2   3  4
年度  季度                    
上半年 第一季度   2   4  2   2  5
    第二季度   4   3  6  77  9
下半年 第三季度   2   4  6   8  9
    第四季度  22  53  6  77  9

### 数据透视---pivot-table
令参数fill_value=0  可以将Nan填充为0，再进行运算
values可以传入多个参数，进行相关的运算

In [44]:
df.insert(1,'pe_2015',6)

In [45]:
df.insert(1,'pe_2016',8)

In [46]:
df.insert(1,'pe_2017',10)

In [47]:
df.head()

,ts_code,pe_2017,pe_2016,pe_2015,symbol,name,area,industry,market,list_date
0,000001.SZ,10,8,6,000001,平安银行,深圳,银行,主板,1991-04-03
1,000002.SZ,10,8,6,000002,万科A,深圳,全国地产,主板,1991-01-29
2,000004.SZ,10,8,6,000004,国农科技,深圳,生物制药,主板,1991-01-14
3,000005.SZ,10,8,6,000005,世纪星源,深圳,环境保护,主板,1990-12-10
4,000006.SZ,10,8,6,000006,深振业A,深圳,区域地产,主板,1992-04-27


#### aggfunc、fill_value、margins参数

In [ ]:
df.pivot_table(values='pe_2017',index='industry',columns='market',aggfunc='sum').head()

In [61]:
df.pivot_table(values='pe_2017',index='industry',columns='market',aggfunc='sum',fill_value=0).head()

market,中小板,主板,创业板,科创板
industry,,,,
IT设备,130,110,120,0
专用机械,280,500,520,70
中成药,220,390,80,0
乳制品,70,50,0,0
互联网,270,190,240,0


In [64]:
df.pivot_table(values=['pe_2017','pe_2016'],index='industry',columns='market',aggfunc='sum',fill_value=0,margins=True)

pe_2016                          pe_2017                         
market       中小板     主板   创业板  科创板    All     中小板     主板   创业板  科创板    All
industry                                                                  
IT设备         104     88    96    0    288     130    110   120    0    360
专用机械         224    400   416   56   1096     280    500   520   70   1370
中成药          176    312    64    0    552     220    390    80    0    690
乳制品           56     40     0    0     96      70     50     0    0    120
互联网          216    152   192    0    560     270    190   240    0    700
仓储物流         104    176    24    0    304     130    220    30    0    380
供气供热          40    184     8    0    232      50    230    10    0    290
保险             0     56     0    0     56       0     70     0    0     70
元器件          496    392   520   48   1456     620    490   650   60   1820
全国地产          16    272     0    0    288      20    340     0    0    360
公共交通           8     56     0    0     64      10     70     0    0     80
公路            16     40     0    0     56      20     50     0    0     70
其他商业          32     48    16    0     96      40     60    20    0    120
其他建材         120     72    32    0    224     150     90    40    0    280
农业综合          64     96    48    0    208      80    120    60    0    260
农用机械          24     40     8    0     72      30     50    10    0     90
农药化肥         120    208    16    0    344     150    260    20    0    430
出版业            8    144    32    0    184      10    180    40    0    230
化学制药         240    384   184   16    824     300    480   230   20   1030
化工原料         416    544   312   32   1304     520    680   390   40   1630
化工机械          48     24     8    0     80      60     30    10    0    100
化纤            56    128    16    0    200      70    160    20    0    250
区域地产          56    456     0    0    512      70    570     0    0    640
医疗保健         104    176   296   88    664     130    220   370  110    830
医药商业          40    160     0    0    200      50    200     0    0    250
半导体           72    104   224   56    456      90    130   280   70    570
商品城            8     16     0    0     24      10     20     0    0     30
商贸代理           8    160     0    0    168      10    200     0    0    210
啤酒             8     48     0    0     56      10     60     0    0     70
园区开发           0    128     0    0    128       0    160     0    0    160
...          ...    ...   ...  ...    ...     ...    ...   ...  ...    ...
种植业           16     72    24    0    112      20     90    30    0    140
空运             8     56     0    0     64      10     70     0    0     80
红黄酒            8     72     0    0     80      10     90     0    0    100
纺织           104    192    16    0    312     130    240    20    0    390
纺织机械          24     40    16    0     80      30     50    20    0    100
综合类           16    240    24    0    280      20    300    30    0    350
航空            32    112    48    0    192      40    140    60    0    240
船舶             0     40    24    0     64       0     50    30    0     80
装修装饰         128     56    24    0    208     160     70    30    0    260
证券            64    272     8    0    344      80    340    10    0    430
超市连锁          32     56     0    0     88      40     70     0    0    110
路桥             0    152     0    0    152       0    190     0    0    190
软件服务         280    352   832  104   1568     350    440  1040  130   1960
软饮料            0     48     0    0     48       0     60     0    0     60
轻工机械          32     16    32    0     80      40     20    40    0    100
运输设备          32    104    24   24    184      40    130    30   30    230
通信设备         248    336   344   16    944     310    420   430   20   1180
造纸            56    144     0    0    200      70    180     0    0    250
酒店餐饮          16     56     0    0     72      20     70     0    0     90
钢加工           56    120     8    0

### 拼接和拆分

#### 与numpy中的比较相似:hstack,vstack,hsplit,vsplit

In [24]:
import numpy as np
t=np.arange(12).reshape(3,4)
t

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [25]:
np.hsplit(t,2)

[array([[0, 1],
        [4, 5],
        [8, 9]]), array([[ 2,  3],
        [ 6,  7],
        [10, 11]])]

In [26]:
x1=np.hsplit(t,2)[0]
x2=np.hsplit(t,2)[1]
x1
x2

array([[0, 1],
       [4, 5],
       [8, 9]])

array([[ 2,  3],
       [ 6,  7],
       [10, 11]])

In [27]:
np.vstack((x1,x2))

array([[ 0,  1],
       [ 4,  5],
       [ 8,  9],
       [ 2,  3],
       [ 6,  7],
       [10, 11]])

In [28]:
np.hstack((x1,x2))

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])